In [1]:
import numpy 
import os
import parmed.amber
import trimesh 

## Nueva implicit Data

In [2]:
### Ingresar Archivo con los resultados de PyGBe y el implicit Data a utilizar
def create_implicit_data(archivo,implicit_data): 
    nuevo = open('new_implicit_data.txt','w+') #Nombre nuevo documento
    nuevo.write('IUPAC, SMILES, dGes, dGcav, dGdisp, dGexper, SASA, Vol\n')
    f = open(implicit_data,'r')
    for linea in f.readlines():
        g=open(archivo,'r')
        listaF=linea.split(',')
        for line in g.readlines():
            listaG=line.split('   E_solv = ')
            e,d=listaG[1].split('.')
            nn = e+'.'+d[:4]
            if listaF[0]==listaG[0]:
                new_line=linea.replace(str(listaF[2][1:]),nn)
                nuevo.write(new_line)
        g.close()
    f.close()
    return nuevo.close()


def lista(direccion):
    lista_de_moleculas=[]
    for dirpath ,dirnames, filenames in os.walk(direccion):
        filenames
        for molecula in filenames:
            if '.crd' in molecula:
                molecula=molecula[:-4]
                lista_de_moleculas.append(molecula)
            
            if '.pqr' in molecula:
                molecula=molecula[:-4]
                lista_de_moleculas.append(molecula)
    return lista_de_moleculas



In [3]:
#dirección de las Carpetas con documentos .pqr
lista_molec = lista('pqr/')
create_implicit_data('resultados eps1.05.txt','data/implicit_data.txt')

## N atoms y Vol

In [4]:
##Ingresar Direccion con los datos en PQR
def N_atoms_and_volume(direccion):
    lista_molec = lista(direccion)
    nuevo = open('data/n_atom.txt','w+')
    explicit = open('data/explicit_data.txt','r')
    for L in explicit.readlines():
        L_split = L.split(',')
        name = L_split[0]
        if name in lista_molec:
            contador=0
            dir_archivo= direccion+'/'+name+'/'+name+'.pqr'
            archivo=open(dir_archivo,'r')
            for linea in archivo.readlines():
                if 'ATOM' in linea:
                    contador +=1
            volumen,area = vert_face(direccion,name)
            nuevo.write(name + '   '+str(contador)+'   '+str(volumen)+'   '+str(area)+' \n')
            archivo.close()
    explicit.close()
    return nuevo.close()

def vert_face(direccion,molec):  #Función que entrega el Volumen y el Area de cada 
    V = numpy.loadtxt(direccion+molec+'/geometry/'+molec+'.vert')
    F = numpy.loadtxt(direccion+molec+'/geometry/'+molec+'.face')
    vert = numpy.zeros(len(V)*3)
    face = numpy.zeros(len(F)*3)
    contador_V = 0
    contador_F = 0
    for i in range(len(V)):
        for j in range(3):
            vert[contador_V]=V[i][j]
            contador_V+=1
    vertices=vert.reshape(len(V),3)

    for k in range(len(F)):
        for l in range(3):
            face[contador_F]=F[k][l] - 1
            contador_F+=1
    faces=face.reshape(len(F),3)
    mesh = trimesh.Trimesh(vertices,
                       faces,process=False)
    return mesh.volume, mesh.area


In [5]:
lista_molec = lista('pqr/')
N_atoms_and_volume('pqr/')